Transform Coords file into video with virtualization

In [9]:
import os
import sys
sys.path.append(os.getenv("PROJECT_PATH"))

from utils.drawpitch import draw_pitch, draw_points_on_pitch,draw_paths_on_pitch
from utils.pitchconfig import SoccerPitchConfiguration

import pandas as pd
import numpy as np
import supervision as sv

df = pd.read_csv("../../data/predicted/lstm_88_comb.csv")
#df = pd.read_csv("../../data/train/train_interpolated_combined.csv")

In [10]:
#Supervision - virtualization
CONFIG = SoccerPitchConfiguration()

edge_annotator = sv.EdgeAnnotator(
    color=sv.Color.from_hex('#00BFFF'),
    thickness=2, edges=CONFIG.edges)
vertex_annotator = sv.VertexAnnotator(
    color=sv.Color.from_hex('#FF1493'),
    radius=8)
vertex_annotator_2 = sv.VertexAnnotator(
    color=sv.Color.from_hex('#00BFFF'),
    radius=8)

In [11]:
import pandas as pd
import numpy as np
import cv2
import os
from tqdm import tqdm

from utils.drawpitch import draw_pitch, draw_points_on_pitch
from utils.pitchconfig import SoccerPitchConfiguration
import supervision as sv

#cols = [col for col in df.columns if col.startswith("true_")]
cols = [col for col in df.columns]
df_pred = df[cols]

# === Prepare output directory ===
output_dir = "frames_lstm88"
os.makedirs(output_dir, exist_ok=True)

# === Pitch configuration ===
pitch_config = SoccerPitchConfiguration()
scale = 0.1
padding = 50
dot_color = sv.Color(255, 215, 0)  # Yellow for predictions

team0_cols = [col for col in cols if "_team_0" in col]
team1_cols = [col for col in cols if "_team_1" in col]
ball_cols = [col for col in cols if "ball" in col]

color_team0 = sv.Color(255, 192, 203)  # Pink
color_team1 = sv.Color(0, 0, 255)      # Blue
color_ball = sv.Color(255, 255, 255)   # White

for _, row in tqdm(df_pred.iterrows(), total=len(df_pred), desc="Rendering frames"):
    frame_index = int(row["frame"])

    team0_points = []
    team1_points = []
    ball_points = []

    for col in team0_cols:
        val = row[col]
        if not isinstance(val, str) or pd.isna(val):
            continue
        try:
            x_str, y_str = val.split(",")
            x, y = float(x_str), float(y_str)
            team0_points.append([x, y])
        except:
            continue

    for col in team1_cols:
        val = row[col]
        if not isinstance(val, str) or pd.isna(val):
            continue
        try:
            x_str, y_str = val.split(",")
            x, y = float(x_str), float(y_str)
            team1_points.append([x, y])
        except:
            continue

    for col in ball_cols:
        val = row[col]
        if not isinstance(val, str) or pd.isna(val):
            continue
        try:
            x_str, y_str = val.split(",")
            x, y = float(x_str), float(y_str)
            ball_points.append([x, y])
        except:
            continue

    if not (team0_points or team1_points or ball_points):
        continue

    pitch = draw_pitch(
        config=pitch_config,
        background_color=sv.Color(34, 139, 34),
        line_color=sv.Color.WHITE,
        scale=scale,
        padding=padding
    )

    if team0_points:
        pitch = draw_points_on_pitch(
            config=pitch_config,
            xy=np.array(team0_points),
            pitch=pitch,
            face_color=color_team0,
            edge_color=sv.Color.BLACK,
            radius=10,
            thickness=2,
            scale=scale,
            padding=padding
        )

    if team1_points:
        pitch = draw_points_on_pitch(
            config=pitch_config,
            xy=np.array(team1_points),
            pitch=pitch,
            face_color=color_team1,
            edge_color=sv.Color.BLACK,
            radius=10,
            thickness=2,
            scale=scale,
            padding=padding
        )

    if ball_points:
        pitch = draw_points_on_pitch(
            config=pitch_config,
            xy=np.array(ball_points),
            pitch=pitch,
            face_color=color_ball,
            edge_color=sv.Color.BLACK,
            radius=7,
            thickness=2,
            scale=scale,
            padding=padding
        )

    cv2.imwrite(os.path.join(output_dir, f"frame_{frame_index:05d}.png"), pitch)

Rendering frames:   0%|          | 0/109 [00:00<?, ?it/s]

Rendering frames: 100%|██████████| 109/109 [00:02<00:00, 46.36it/s]


In [14]:
import imageio

frame_folder = "frames_lstm88"
frame_files = sorted([f for f in os.listdir(frame_folder) if f.endswith(".png")])

images = []
for filename in frame_files:
    filepath = os.path.join(frame_folder, filename)
    images.append(imageio.imread(filepath))

imageio.mimsave("../../data/animations/lstm88_frames.gif", images, fps=6.06)